In [0]:
df_cust = spark.table("jpm.kunal_bronze.customer_bronze")
df_trans = spark.table("jpm.kunal_silver.silver_transaction")

In [0]:
%python
df_joined = df_cust.join(
    df_trans, 
    df_cust.cust_id == df_trans.cust_id, 
    "inner"
).select(
    df_cust.cust_id,
    'name',
    'age',
    'gender',
    'birthday',
    'zip',
    df_cust.city,
    'start_date',
    'end_date',
    'txn_id',
    'date',
    'year',
    'month',
    'day',
    'expense_type',
    'amt'
)

In [0]:
df_joined.createOrReplaceTempView("silver_joined")

In [0]:
%sql
create or replace table jpm.kunal_gold.all_records as 
select * from silver_joined

In [0]:
%sql
create or replace view jpm.kunal_gold.city_totals as 
select city, sum(amt) as total_sum from jpm.kunal_gold.all_records group by all

In [0]:
%sql
select * from jpm.kunal_gold.city_totals

In [0]:
%sql
create or replace view jpm.kunal_gold.cust_name_rank as
select name, city, RANK() OVER (partition by city ORDER BY SUM(amt) DESC) AS rank from jpm.kunal_gold.all_records group by name, city  

In [0]:
%sql
select * from jpm.kunal_gold.cust_name_rank